In [3]:
import zmq
import datetime
import plotly.graph_objects as go

```
conda install plotly ipywidgets
jupyter labextension install jupyterlab-plotly
jupyter labextension install @jupyter-widgets/jupyterlab-manager
jupyter labextension install plotlywidget
```

In [4]:
symbol = 'SYMBOL'

In [5]:
fig = go.FigureWidget()
fig.add_scatter()
fig

FigureWidget({
    'data': [{'type': 'scatter', 'uid': 'f778d2a2-52ef-402b-9dd0-63ab1398863b'}], 'layout': {'template': '...'}
})

In [6]:
context = zmq.Context()
socket = context.socket(zmq.SUB)
socket.connect('tcp://0.0.0.0:5555')

socket.setsockopt_string(zmq.SUBSCRIBE, 'SYMBOL')

In [10]:
times = []
prices = []

for _ in range(50):
    msg = socket.recv_string()
    t = datetime.datetime.now()
    times.append(t)
    _, price = msg.split()
    prices.append(float(price))
    fig.data[0].x = times
    fig.data[0].y = prices
    

In [12]:
fig = go.FigureWidget()
fig.add_scatter(name='SYMBOL')
fig.add_scatter(name='SMA1', line=dict(width=1, dash='dot'),mode='lines+markers')
fig.add_scatter(name='SMA2', line=dict(width=1, dash='dash'),mode='lines+markers')
fig

FigureWidget({
    'data': [{'name': 'SYMBOL', 'type': 'scatter', 'uid': '914754f2-582e-4b22-b225-12b95e6fea49'},
             {'line': {'dash': 'dot', 'width': 1},
              'mode': 'lines+markers',
              'name': 'SMA1',
              'type': 'scatter',
              'uid': 'f330a26f-dc06-4ec8-891f-37d2c81ab0eb'},
             {'line': {'dash': 'dash', 'width': 1},
              'mode': 'lines+markers',
              'name': 'SMA2',
              'type': 'scatter',
              'uid': '28957684-8d75-42e9-9428-34a2b1f3e069'}],
    'layout': {'template': '...'}
})

In [14]:
import pandas as pd
df = pd.DataFrame()

for _ in range(75):
    msg = socket.recv_string()
    t = datetime.datetime.now()
    sym, price = msg.split()
    new_row = pd.DataFrame({sym: [price]}, index=[t])
    
    df = pd.concat([df, new_row])
    df['SMA1'] = df[sym].rolling(5).mean()
    df['SMA2'] = df[sym].rolling(10).mean()
    fig.data[0].x = df.index
    fig.data[1].x = df.index
    fig.data[2].x = df.index
    fig.data[0].y = df[sym]
    fig.data[1].y = df['SMA1']
    fig.data[2].y = df['SMA2']
    